# Python Text Mining Assignment 3

In this assignment you will explore text message data and create models to predict if a message is spam or not. 

In [1]:
import pandas as pd
import numpy as np

# Read in text messages with labels
spam_data = pd.read_csv('spam.csv')

spam_data['target'] = np.where(spam_data['target']=='spam',1,0)
spam_data.head(10)

,text,target
0,"Go until jurong point, crazy.. Available only ...",0
1,Ok lar... Joking wif u oni...,0
2,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,U dun say so early hor... U c already then say...,0
4,"Nah I don't think he goes to usf, he lives aro...",0
5,FreeMsg Hey there darling it's been 3 week's n...,1
6,Even my brother is not like to speak with me. ...,0
7,As per your request 'Melle Melle (Oru Minnamin...,0
8,WINNER!! As a valued network customer you have...,1
9,Had your mobile 11 months or more? U R entitle...,1


In [2]:
from sklearn.model_selection import train_test_split

# Split data into test and training set
X_train, X_test, y_train, y_test = train_test_split(spam_data['text'], 
                                                    spam_data['target'], 
                                                    random_state=0)

### Question 1
What percentage of the documents in `spam_data` are spam?

*This function should return a float, the percent value (i.e. $ratio * 100$).*

In [3]:
def answer_one():
    
    return np.mean(spam_data['target']) * 100 

In [4]:
answer_one()

13.406317300789663

### Question 2

Fit the training data `X_train` using a Count Vectorizer with default parameters.

What is the longest token in the vocabulary?

*This function should return a string.*

In [5]:
from sklearn.feature_extraction.text import CountVectorizer

def answer_two():
    
    # CountVectorizer converts a collection of text to a matrix of token counts
    vect = CountVectorizer().fit(X_train)
    # Get token name
    tokens = vect.get_feature_names()
    
    return max(tokens, key = len) #Your answer here

In [6]:
answer_two()

'com1win150ppmx3age16subscription'

### Question 3

Fit and transform the training data `X_train` using a Count Vectorizer with default parameters.

Next, fit a fit a multinomial Naive Bayes classifier model with smoothing `alpha=0.1`. Find the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [7]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score

def answer_three():
    
    vect = CountVectorizer().fit(X_train)
    X_train_vect = vect.transform(X_train)
    
    # Train Naive Bayes
    clfNB = MultinomialNB(alpha=0.1).fit(X_train_vect, y_train)
    # Predict test set y
    y_pred = clfNB.predict(vect.transform(X_test))
    
    # Return AUC score
    return roc_auc_score(y_test, y_pred) #Your answer here

In [8]:
answer_three()

0.9720812182741116

### Question 4

Fit and transform the training data `X_train` using a Tfidf Vectorizer with default parameters.

What 20 features have the smallest tf-idf and what 20 have the largest tf-idf?

Put these features in a two series where each series is sorted by tf-idf value and then alphabetically by feature name. The index of the series should be the feature name, and the data should be the tf-idf.

The series of 20 features with smallest tf-idfs should be sorted smallest tfidf first, the list of 20 features with largest tf-idfs should be sorted largest first. 

*This function should return a tuple of two series
`(smallest tf-idfs series, largest tf-idfs series)`.*

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

def answer_four():
    
    # TfidfVectorizer converts a collection of text to a matrix of token tf-idf
    # (term frequency-inverse document frequency), which reflects how important 
    # a word is to a document
    vect = TfidfVectorizer().fit(X_train)
    X_train_vect = vect.transform(X_train)
    # Get the max tf-idf for each token
    tf_idf = X_train_vect.max(0).toarray()[0]
    # Get rank for each token according to tf-idf
    sorted_tfidf_index = tf_idf.argsort()

    features = np.array(vect.get_feature_names())
    smallest = pd.Series(tf_idf[sorted_tfidf_index][:20], 
                         index = features[sorted_tfidf_index][:20])
    largest = pd.Series(tf_idf[sorted_tfidf_index][:-21:-1], 
                        index = features[sorted_tfidf_index][:-21:-1])
    
    return (smallest, largest) 

In [10]:
answer_four()

(sympathetic     0.074475
 healer          0.074475
 aaniye          0.074475
 dependable      0.074475
 companion       0.074475
 listener        0.074475
 athletic        0.074475
 exterminator    0.074475
 psychiatrist    0.074475
 pest            0.074475
 determined      0.074475
 chef            0.074475
 courageous      0.074475
 stylist         0.074475
 psychologist    0.074475
 organizer       0.074475
 pudunga         0.074475
 venaam          0.074475
 diwali          0.091250
 mornings        0.091250
 dtype: float64,
 146tf150p    1.000000
 havent       1.000000
 home         1.000000
 okie         1.000000
 thanx        1.000000
 er           1.000000
 anything     1.000000
 lei          1.000000
 nite         1.000000
 yup          1.000000
 thank        1.000000
 ok           1.000000
 where        1.000000
 beerage      1.000000
 anytime      1.000000
 too          1.000000
 done         1.000000
 645          1.000000
 tick         0.980166
 blank        0.932702
 dt

### Question 5

Fit and transform the training data `X_train` using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **3**.

Then fit a multinomial Naive Bayes classifier model with smoothing `alpha=0.1` and compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [11]:
def answer_five():
    
    vect = TfidfVectorizer(min_df=3).fit(X_train)
    X_train_vect = vect.transform(X_train)
    X_test_vect = vect.transform(X_test)

    clfNB = MultinomialNB(alpha = 0.1).fit(X_train_vect, y_train)
    y_pred = clfNB.predict(X_test_vect)
    
    return roc_auc_score(y_test, y_pred) 

In [12]:
answer_five()

0.9416243654822335

### Question 6

What is the average length of documents (number of characters) for not spam and spam documents?

*This function should return a tuple (average length not spam, average length spam).*

In [13]:
def answer_six():
    
    nonspam_text = spam_data[spam_data['target']==0]['text']
    spam_text = spam_data[spam_data['target']==1]['text']
    
    # List comprehension to get length for each text
    avg_len_nonspam = np.mean([len(t) for t in nonspam_text])
    avg_len_spam = np.mean([len(t) for t in spam_text])
    
    return (avg_len_nonspam, avg_len_spam) 

In [14]:
answer_six()

(71.02362694300518, 138.8661311914324)

<br>
<br>
The following function has been provided to help you combine new features into the training data:

In [15]:
def add_feature(X, feature_to_add):
    """
    Returns sparse feature matrix with added feature.
    feature_to_add can also be a list of features.
    """
    from scipy.sparse import csr_matrix, hstack
    return hstack([X, csr_matrix(feature_to_add).T], 'csr')

### Question 7

Fit and transform the training data X_train using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **5**.

Using this document-term matrix and an additional feature, **the length of document (number of characters)**, fit a Support Vector Classification model with regularization `C=10000`. Then compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [16]:
from sklearn.svm import SVC

def answer_seven():
    vect = TfidfVectorizer(min_df=5).fit(X_train)
    X_train_vect = vect.transform(X_train)
    X_test_vect = vect.transform(X_test)
    
    # Add text length to features
    len_train = [len(t) for t in X_train]
    len_test = [len(t) for t in X_test]
    X_train_vect = add_feature(X_train_vect, len_train)
    X_test_vect = add_feature(X_test_vect, len_test)
    
    # Train support vector machine
    clfSVM = SVC(C = 10000).fit(X_train_vect, y_train)
    y_pred = clfSVM.predict(X_test_vect)    
    
    return roc_auc_score(y_test, y_pred) 

In [17]:
answer_seven()

0.9661689557407943

### Question 8

What is the average number of digits per document for not spam and spam documents?

*This function should return a tuple (average # digits not spam, average # digits spam).*

In [18]:
def count_digit(text):
    count = 0
    for w in text:
        if w.isdigit():
            count += 1
    return count   

In [19]:
def answer_eight():
    
    nonspam_text = spam_data[spam_data['target']==0]['text']
    spam_text = spam_data[spam_data['target']==1]['text']
    
    nonspam_digits = np.mean([count_digit(t) for t in nonspam_text])
    spam_digits = np.mean([count_digit(t) for t in spam_text])
    
    return (nonspam_digits, spam_digits) 

In [20]:
answer_eight()

(0.2992746113989637, 15.759036144578314)

### Question 9

Fit and transform the training data `X_train` using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **word n-grams from n=1 to n=3** (unigrams, bigrams, and trigrams).

Using this document-term matrix and the following additional features:
* the length of document (number of characters)
* **number of digits per document**

fit a Logistic Regression model with regularization `C=100`. Then compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [21]:
from sklearn.linear_model import LogisticRegression

def answer_nine():     
    
    # Now the tokens include word bigrams and trigrams, in addition to unigrams
    vect = TfidfVectorizer(min_df=5, ngram_range=(1,3)).fit(X_train)
    X_train_vect = vect.transform(X_train)
    X_test_vect = vect.transform(X_test)
    
    # Add text length to features
    len_train = [len(t) for t in X_train]
    len_test = [len(t) for t in X_test]
    X_train_vect = add_feature(X_train_vect, len_train)
    X_test_vect = add_feature(X_test_vect, len_test)
    
    # Add number of digits to features
    digit_train = [count_digit(t) for t in X_train]
    digit_test = [count_digit(t) for t in X_test]
    X_train_vect = add_feature(X_train_vect, digit_train)
    X_test_vect = add_feature(X_test_vect, digit_test)
    
    # Train logistic regression model
    clfLR = LogisticRegression(C = 100, max_iter=1000).fit(X_train_vect, y_train)
    y_pred = clfLR.predict(X_test_vect)    
    
    return roc_auc_score(y_test, y_pred)

In [22]:
answer_nine()

0.9678709064054463

### Question 10

What is the average number of non-word characters (anything other than a letter, digit or underscore) per document for not spam and spam documents?

*Hint: Use `\w` and `\W` character classes*

*This function should return a tuple (average # non-word characters not spam, average # non-word characters spam).*

In [23]:
def count_non_word(text):
    count = 0
    for w in text:
        if w in r'[\W]':
            count += 1
    return count

In [24]:
import re
def answer_ten():
    
    nonspam_text = spam_data[spam_data['target']==0]['text']
    spam_text = spam_data[spam_data['target']==1]['text']
    
    nonword_nonspam = np.mean([len(re.findall('\W', t)) for t in nonspam_text])
    nonword_spam = np.mean([len(re.findall('\W', t)) for t in spam_text])
    
    return (nonword_nonspam, nonword_spam) 

In [25]:
answer_ten()

(17.29181347150259, 29.041499330655956)

### Question 11

Fit and transform the training data X_train using a Count Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **character n-grams from n=2 to n=5.**

To tell Count Vectorizer to use character n-grams pass in `analyzer='char_wb'` which creates character n-grams only from text inside word boundaries. This should make the model more robust to spelling mistakes.

Using this document-term matrix and the following additional features:
* the length of document (number of characters)
* number of digits per document
* **number of non-word characters (anything other than a letter, digit or underscore.)**

fit a Logistic Regression model with regularization C=100. Then compute the area under the curve (AUC) score using the transformed test data.

Also **find the 10 smallest and 10 largest coefficients from the model** and return them along with the AUC score in a tuple.

The list of 10 smallest coefficients should be sorted smallest first, the list of 10 largest coefficients should be sorted largest first.

The three features that were added to the document term matrix should have the following names should they appear in the list of coefficients:
['length_of_doc', 'digit_count', 'non_word_char_count']

*This function should return a tuple `(AUC score as a float, smallest coefs list, largest coefs list)`.*

In [26]:
def answer_eleven():     
    
    # Now we use character 2- to 5-grams by passing in analyzer='char_wb'
    vect = CountVectorizer(min_df=5, ngram_range=(2,5), 
                           analyzer='char_wb').fit(X_train)
    X_train_vect = vect.transform(X_train)
    X_test_vect = vect.transform(X_test)
    
    # Add text length to features
    len_train = [len(t) for t in X_train]
    len_test = [len(t) for t in X_test]
    X_train_vect = add_feature(X_train_vect, len_train)
    X_test_vect = add_feature(X_test_vect, len_test)
    
    # Add number of digits to features
    digit_train = [count_digit(t) for t in X_train]
    digit_test = [count_digit(t) for t in X_test]
    X_train_vect = add_feature(X_train_vect, digit_train)
    X_test_vect = add_feature(X_test_vect, digit_test)
    
    # Add non-word count to features
    nonword_train = [len(re.findall('\W', t)) for t in X_train]
    nonword_test = [len(re.findall('\W', t)) for t in X_test]
    X_train_vect = add_feature(X_train_vect, nonword_train)
    X_test_vect = add_feature(X_test_vect, nonword_test)    
    
    # Train logistic regression
    clfLR = LogisticRegression(C = 100, max_iter=1000).fit(X_train_vect, y_train)
    y_pred = clfLR.predict(X_test_vect)    
    
    # Get feature importance
    sorted_coef_index = clfLR.coef_[0].argsort()
    features = np.array(vect.get_feature_names())
    features = np.append(features, 
                         ['length_of_doc', 'digit_count', 'non_word_char_count'])
    
    return (roc_auc_score(y_test, y_pred),
           features[sorted_coef_index][:10], 
            features[sorted_coef_index][:-11:-1])    

In [27]:
answer_eleven()

(0.9780231906694056,
 array([' i', 'ca', '..', '. ', 'pe', ' go', ' m', 'if', 'us', 'go'],
       dtype='<U19'),
 array(['digit_count', 'ia', ' r', 'xt', 'ne', 'co', ' ba', ' x', '46',
        'ian '], dtype='<U19'))

In [28]:
def answer_twelve():     
    
    # Try the same using word 1- to 3-grams
    vect = CountVectorizer(min_df=3, ngram_range=(1,3)).fit(X_train)
    X_train_vect = vect.transform(X_train)
    X_test_vect = vect.transform(X_test)
    
    # Add text length to features
    len_train = [len(t) for t in X_train]
    len_test = [len(t) for t in X_test]
    X_train_vect = add_feature(X_train_vect, len_train)
    X_test_vect = add_feature(X_test_vect, len_test)
    
    # Add number of digits to features
    digit_train = [count_digit(t) for t in X_train]
    digit_test = [count_digit(t) for t in X_test]
    X_train_vect = add_feature(X_train_vect, digit_train)
    X_test_vect = add_feature(X_test_vect, digit_test)
    
    # Add non-word count to features
    nonword_train = [len(re.findall('\W', t)) for t in X_train]
    nonword_test = [len(re.findall('\W', t)) for t in X_test]
    X_train_vect = add_feature(X_train_vect, nonword_train)
    X_test_vect = add_feature(X_test_vect, nonword_test)    
    
    # Train logistic regression
    clfLR = LogisticRegression(C = 100, max_iter=1000).fit(X_train_vect, y_train)
    y_pred = clfLR.predict(X_test_vect)    
    
    # Get feature importance
    sorted_coef_index = clfLR.coef_[0].argsort()
    features = np.array(vect.get_feature_names())
    features = np.append(features, 
                         ['length_of_doc', 'digit_count', 'non_word_char_count'])
    
    return (roc_auc_score(y_test, y_pred),
           features[sorted_coef_index][:10], 
            features[sorted_coef_index][:-11:-1])   

In [29]:
answer_twelve()

(0.9712748077347504,
 array(['all', 'if', 'got', 'call', 'my', 'hey', 'say', 'just', 'tomorrow',
        '1st'], dtype='<U39'),
 array(['text', 'sale', 'tones', 'but no', 'reply', 'girls', 'sms',
        'content', 'free', 'for'], dtype='<U39'))

## Reflections
It is interesting to find that character n-grams used in `answer_eleven()` works significantly better than word n-grams used in `answer_twelve()`, with all other conditions being the same. The most and least coefficients for `answer_eleven()` are also somewhat harder to interpret. The fact that character n-grams work better might have to do with spam email containing a lot of non-standard spellings, perhaps in order to bypass the spam filtering systems, as shown in the samples below. The non-standard spellings might be picked up more easily using character n-grams.

In [30]:
spam_data[spam_data['target']==1]['text'][:10]

2     Free entry in 2 a wkly comp to win FA Cup fina...
5     FreeMsg Hey there darling it's been 3 week's n...
8     WINNER!! As a valued network customer you have...
9     Had your mobile 11 months or more? U R entitle...
11    SIX chances to win CASH! From 100 to 20,000 po...
12    URGENT! You have won a 1 week FREE membership ...
15    XXXMobileMovieClub: To use your credit, click ...
19    England v Macedonia - dont miss the goals/team...
34    Thanks for your subscription to Ringtone UK yo...
42    07732584351 - Rodger Burns - MSG = We tried to...
Name: text, dtype: object